In [1]:
import pandas as pd
from pandas.io.json import json_normalize 
import os
import plotly.express as px
import json 

#dataset = "CIFAR100"
dataset = "FASHION"
all_subdirs = [os.path.join(dataset,d) for d in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, d)) and d != "cifar-100-python"]
print(all_subdirs)
#latest_folder = max(all_subdirs, key=os.path.getmtime)
latest_folder = os.path.join(dataset,"24-11-2020-15:02:28")

data = []
with open(os.path.join(latest_folder, "results.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

#print(data)
df = json_normalize(data)
df = df.applymap(lambda x: x[0] if (isinstance(x, list) and len(x) == 1) else x)

def nice_name(row):
    if row["model"] == "GNCLClassifier":
        if row["l_reg"] == 0:
            return "Ind."
        else:
            return u"GNLC \u03BB = {}".format(row["l_reg"]) 
    elif row["model"] == "BaggingClassifier":
        return "Bagging"
    elif row["model"] == "E2EEnsembleClassifier":
        return "E2E"
    elif row["model"] == "Model":
        return "Single"
    elif row["model"] == "SMCLClassifier":
        return "SMCL"
    elif row["model"] == "GradientBoostedNets":
        return "GB"
    elif row["model"] == "SnapshotEnsembleClassifier":
        return "SE"
    else:
        return row["model"]

def base_type(row):
    if "binary" in row["base_estimator"]:
        mtype = "binary"
    else:
        mtype = "float"

    if "small" in row["base_estimator"]:
        size = "small"
    else:
        size = "large"

    return row["base_estimator"].split("_")[0] + " " + size + " " + mtype

print(df.columns)     
df["nice_name"] = df.apply(nice_name, axis=1)
df["model_type"] = df.apply(base_type, axis=1)
df["complexity"] = df.apply(lambda x : x["scores.params"] if "float" in x["model_type"] else x["scores.params"] / 32.0, axis=1)
df["scores.avg_accurcay_test"] = df["scores.mean_test_avg_accurcay"]
df["scores.accuracy_test"] = 100.0*df["scores.mean_test_accuracy"]
df = df.round(decimals = 3)

print("{} experiments done".format(len(df)))
cols = df.columns
cols = ["nice_name", "model_type", "mode", "scores.mean_test_accuracy", "scores.params", "scores.mean_train_diversity"] + list(set(cols) - set(["nice_name", "model_type", "mode", "scores.mean_accuracy_test", "scores.params", "scores.mean_train_diversity"]))
df = df.reindex(cols, axis=1)
df
#dfbinary = df[df["model_type"] == "binary"]
#dffloat = df[df["model_type"] == "float"]  
#dffloat


['FASHION/24-11-2020-15:02:28', 'FASHION/27-11-2020-22:14:17', 'FASHION/FashionMNIST', 'FASHION/27-11-2020-15:51:17', 'FASHION/19-11-2020-22:25:09']
Index(['base_estimator', 'classifier', 'device', 'eval_every', 'experiment_id',
       'loss_function', 'model', 'n_estimators', 'out_path', 'store_every',
       'test_data', 'train_data', 'use_amp', 'verbose', 'loader.batch_size',
       'loader.num_workers', 'loader.pin_memory', 'optimizer.betas',
       'optimizer.epochs', 'optimizer.eps', 'optimizer.lr', 'optimizer.method',
       'scheduler.gamma', 'scheduler.method', 'scheduler.step_size',
       'scores.fit_time', 'scores.mean_fit_time', 'scores.mean_params',
       'scores.mean_test_accuracy', 'scores.mean_test_avg_accurcay',
       'scores.mean_test_avg_loss', 'scores.mean_test_loss',
       'scores.mean_test_diversity', 'scores.params', 'scores.std_fit_time',
       'scores.std_params', 'scores.std_test_accuracy',
       'scores.std_test_avg_accurcay', 'scores.std_test_avg_loss'

,nice_name,model_type,mode,scores.mean_test_accuracy,scores.params,scores.mean_train_diversity,scores.std_fit_time,test_data,scores.test_accuracy,scores.mean_test_avg_loss,...,scores.test_loss,scores.test_diversity,classifier,scores.mean_test_accuracy,scores.std_test_loss,store_every,scores.std_test_diversity,optimizer.betas,l_reg,scores.test_avg_accurcay
0,StackingClassifier,simpleresnet large float,NaN,94.746,11344180,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.746,6.192,...,0.354,8.317,stacking_classifier_{'model_type': 'float'},94.746,0.0,0,0.0,"[0.9, 0.999]",NaN,10.126
1,Bagging,simpleresnet large float,NaN,94.961,10675360,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.961,0.302,...,0.212,0.081,NaN,94.961,0.0,0,0.0,"[0.9, 0.999]",NaN,93.677
2,GB,simpleresnet large float,NaN,94.199,10675360,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.199,2.119,...,1.199,3.413,NaN,94.199,0.0,0,0.0,"[0.9, 0.999]",NaN,22.014
3,SE,simpleresnet large float,NaN,94.395,10675360,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.395,2.419,...,0.220,1.544,NaN,94.395,0.0,0,0.0,"[0.9, 0.999]",NaN,70.434
4,E2E,simpleresnet large float,NaN,94.990,10675360,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.990,19.557,...,0.285,6.061,NaN,94.990,0.0,0,0.0,"[0.9, 0.999]",NaN,28.292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,SE,simpleresnet large float,NaN,94.170,10675360,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,94.170,2.960,...,0.221,1.807,NaN,94.170,0.0,0,0.0,"[0.9, 0.999]",NaN,67.630
68,Single,simpleresnet small binary,NaN,91.377,75211,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,91.377,0.000,...,0.239,0.000,NaN,91.377,0.0,1,0.0,"[0.9, 0.999]",NaN,0.000
69,Single,simpleresnet small float,NaN,93.369,74954,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,93.369,0.000,...,0.187,0.000,NaN,93.369,0.0,1,0.0,"[0.9, 0.999]",NaN,0.000
70,Single,simpleresnet large binary,NaN,93.857,667979,NaN,0.0,Dataset FashionMNIST\n Number of datapoints...,93.857,0.000,...,0.191,0.000,NaN,93.857,0.0,1,0.0,"[0.9, 0.999]",NaN,0.000


In [2]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

fig = make_subplots(rows=4, cols=3, subplot_titles=["Small Binary", "Small Float", "Large Float"], horizontal_spacing = 0.03, vertical_spacing = 0.02)
    
methods_order = ["SMCL", "GB", "SE", "Bagging", "Ind.", "GNLC λ = 0.1", "GNLC λ = 0.2", "GNLC λ = 0.3", "GNLC λ = 0.4" ,"GNLC λ = 0.5", "GNLC λ = 0.6", "GNLC λ = 0.7", "GNLC λ = 0.8", "GNLC λ = 0.9", "E2E"]

shapes = []
for i, mtype in enumerate(["simpleresnet small binary", "simpleresnet small float", "simpleresnet large float"]):
    filteredDF = df.loc[ df["model_type"] == mtype ]
    dff = filteredDF.loc[ (filteredDF["n_estimators"] == 16)]
    
    accuracy_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.accuracy_test"].to_numpy()[0]
    trainloss_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.loss_train"].to_numpy()[0]

    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.accuracy_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.accuracy_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=1,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.diversity_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.diversity_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=2,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.avg_accurcay_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.avg_accurcay_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=3,col=i+1)
    #fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.loss_train"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.loss_train"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=4,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.loss_train"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.test_loss"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=4,col=i+1)
    
    fig.update_yaxes(row=1, col=i+1, dtick=10, tickmode = 'linear',tick0 = 0)
    fig.update_yaxes(row=2, col=i+1, type="log")
    fig.update_yaxes(row=3, col=i+1, dtick=10, tickmode = 'linear',tick0 = 0)
    fig.update_yaxes(row=4, col=i+1, type="log")
        
    fig.update_xaxes(row=1, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=2, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=3, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=4, col=i+1, categoryarray=methods_order, tickangle=45)
    
    shapes.extend([
        dict(
            type='line',xref='x'+str(i+1),yref='y'+str(i+1),x0=-0.5,y0=accuracy_single,x1=len(methods_order)-1+0.5,y1=accuracy_single,
            line=dict(color='Dark Grey',width=1,dash="dashdot")
        ),
        dict(
            type='line',xref='x'+str(i+10),yref='y'+str(i+10),x0=-0.5,y0=trainloss_single,x1=len(methods_order)-1+0.5,y1=trainloss_single,
            line=dict(color='Dark Grey',width=1,dash="dashdot")
        )
    ])
    

fig.update_yaxes(title_text="Accuracy", row=1, col=1, title_standoff = 25, title_font = {"size": 14})
fig.update_yaxes(title_text="Diversity", row=2, col=1, title_font = {"size": 14})
fig.update_yaxes(title_text="Average accuracy", row=3, col=1, title_font = {"size": 14})
fig.update_yaxes(title_text="Train loss", row=4, col=1, title_font = {"size": 14})


fig.update_layout(
    shapes=shapes,
    legend=dict(orientation="h",yanchor="bottom",y=-0.1,xanchor="left",x=0.1),
    margin={'l': 5, 'r': 20, 't': 20, 'b': 5},
    height=700, width=1200
)

fig.write_image("images/fashion.pdf")
fig.show()


KeyError: 'scores.loss_train'